In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# Define the true function
def true_function(x):
    """y = sin(π*x/2) + x/2"""
    return np.sin((np.pi/2) * x) + x/2


def linear_interpolate_sid(x_data, y_data, x_query):
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)
    x_query = np.asarray(x_query)
    
    if not np.all(x_data[:-1] <= x_data[1:]):
        sort_indices = np.argsort(x_data)
        x_data = x_data[sort_indices]
        y_data = y_data[sort_indices]
    
    y_interp = np.zeros_like(x_query, dtype=float)
    
    for idx, x in enumerate(x_query):
        if x <= x_data[0]:
            y_interp[idx] = y_data[0]
            continue
        if x >= x_data[-1]:
            y_interp[idx] = y_data[-1]
            continue
        for i in range(len(x_data) - 1):
            if x_data[i] <= x <= x_data[i + 1]:
                x0 = x_data[i]
                x1 = x_data[i + 1]
                y0 = y_data[i]
                y1 = y_data[i + 1]
                y_interp[idx] = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
                break
    return y_interp


# READ DATA
file_path = r"D:\\Physics\\Computational Physics and Astrophysics\\Interpolation\\function_data.txt"

x_data = []
y_data = []

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        parts = line.split()
        if len(parts) >= 2:
            x_data.append(float(parts[0]))
            y_data.append(float(parts[1]))

x_data = np.array(x_data)
y_data = np.array(y_data)

# HIGH-RES GRID AND INTERPOLANTS
x_eval = np.linspace(x_data.min(), x_data.max(), len(x_data) * 10 - 9)
y_linear = linear_interpolate_sid(x_data, y_data, x_eval)
cubic_spline = CubicSpline(x_data, y_data, bc_type='natural')
y_cubic = cubic_spline(x_eval)
y_true = true_function(x_eval)

# ============================================================================
# PLOT 1: TRUE FUNCTION, LINEAR SPLINE, AND CUBIC SPLINE
# ============================================================================

fig, ax1 = plt.subplots(figsize=(14, 8))

ax1.plot(x_eval, y_true, 'k-', linewidth=3, label='True Function', zorder=2)
ax1.plot(x_eval, y_linear, color='green', linewidth=2.5, linestyle='--', 
         label='Linear Spline (SID)', zorder=2, alpha=0.8)
ax1.plot(x_eval, y_cubic, color='red', linewidth=2.5, linestyle='-', 
         label='Cubic Spline', zorder=2, alpha=0.8)

ax1.scatter(x_data, y_data, s=120, color='blue', marker='D', 
            edgecolors='black', linewidth=2, zorder=5, label='Original Data Points')

ax1.set_xlabel('x', fontsize=12, fontweight='bold')
ax1.set_ylabel('y', fontsize=12, fontweight='bold')
ax1.set_title('True Function vs Linear Spline vs Cubic Spline', fontsize=13, fontweight='bold')
ax1.set_xlim([0, 10])
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11, loc='best', framealpha=0.95)

plt.tight_layout()
plt.savefig('interpolation_comparison.png', dpi=150, bbox_inches='tight')
print("\n✓ Plot 1 saved as 'interpolation_comparison.png'")
plt.show()
